# Coulomb gauge transformation

### Simulation parameters

In [1]:
import numpy as np

# hbar * c [GeV * fm]
hbarc = 0.197326 

# Simulation box 
L = 2      
N = 64
# N = 512  
tau_sim = 0.3   
DTS = 4     

# Glasma fields
su_group = 'su3'
# Qs = 2
Qs = 0.01
g = 1

ns = 50    
factor = 0.8        
g2mu = Qs / factor     
# g = np.pi * np.sqrt(1 / np.log(Qs / 0.2))          		
mu = g2mu / g**2          	
ir = 0.1 * g**2 * mu         
uv = 10.0       

nevents = 1

### Environment variables

In [2]:
import os

# I need to add this line to ask resources from a specific GPU, which is free. Our GPU server has no queing system
# os.environ["CUDA_VISIBLE_DEVICES"]="1, 6"

os.environ["MY_NUMBA_TARGET"] = "cuda"
# os.environ["MY_NUMBA_TARGET"] = "numba"
os.environ["PRECISION"] = "double"
os.environ['GAUGE_GROUP'] = su_group

# Import relevant modules
import sys
sys.path.append('..')

# Glasma modules
import curraun.core as core
import curraun.mv as mv
import curraun.initial as initial
initial.DEBUG = False

import curraun.su as su
import curraun.lattice as latt
from curraun.numba_target import use_cuda
if use_cuda:
    from numba import cuda

import curraun.su as su
import curraun.coulomb as coulomb
coulomb.DEBUG = True
coulomb.TRANS_GLASMA = True

import curraun.energy as energy

Using CUDA
Using SU(3)
Using double precision


### Simulate glasma

In [3]:
import pickle
from tqdm import tqdm

# Simulation routine
# def simulate(): 

# Derived parameters
a = L / N
E0 = N / L * hbarc
DT = 1.0 / DTS
maxt = int(tau_sim / a * DTS)

# Initialize Glasma fields
s = core.Simulation(N, DT, g)
va = mv.wilson(s, mu=mu / E0, m=ir / E0, uv=uv / E0, num_sheets=ns)
vb = mv.wilson(s, mu=mu / E0, m=ir / E0, uv=uv / E0, num_sheets=ns)
initial.init(s, va, vb)

# if use_cuda:
#     s.copy_to_device()

# with tqdm(total=maxt) as pbar:
#     for t in range(maxt):      
#         #TODO: evolve the glasma fields, at the moment consider only 1 evolution step

#         # Evolve Glasma fields
#         core.evolve_leapfrog(s)

#         pbar.update(1)

core.evolve_leapfrog(s)

# ed_glasma = energy.Energy(s)
# ed_glasma.compute(s)
# print('ed_glasma =', ed_glasma.energy_density)

c = coulomb.CoulombGaugeTransf(s)

coulomb.init_gauge_transf(c)
coulomb.iter_gauge_transf(c)

# ed_coul = energy.Energy(c)
# ed_coul.compute(c)
# print('ed_coul =', ed_coul.energy_density)

if use_cuda:
    cuda.current_context().deallocations.clear()

alpha: 0.019261093311212417
iter: 1 theta: 1.0378698294632163e-05 accuracy: 0
iter: 2 theta: 9.342799840803176e-06 accuracy: -1
iter: 3 theta: 8.440316722768105e-06 accuracy: -1
iter: 4 theta: 7.652227626597019e-06 accuracy: -1
iter: 5 theta: 6.962376950449859e-06 accuracy: -1
iter: 6 theta: 6.357027783497129e-06 accuracy: -1
iter: 7 theta: 5.824486312678166e-06 accuracy: -1
iter: 8 theta: 5.35478604012747e-06 accuracy: -1
iter: 9 theta: 4.939422109526684e-06 accuracy: -1
iter: 10 theta: 4.5711276357877454e-06 accuracy: -1
iter: 11 theta: 4.243685263042298e-06 accuracy: -1
iter: 12 theta: 3.951768285503145e-06 accuracy: -1
iter: 13 theta: 3.690806591425716e-06 accuracy: -1
iter: 14 theta: 3.4568734629202856e-06 accuracy: -1
iter: 15 theta: 3.2465899093411236e-06 accuracy: -1
iter: 16 theta: 3.057043750686552e-06 accuracy: -1
iter: 17 theta: 2.88572111758225e-06 accuracy: -1
iter: 18 theta: 2.7304484107137665e-06 accuracy: -1
iter: 19 theta: 2.5893430772770905e-06 accuracy: -1
iter: 20 